In [155]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd
import config
import build_dataset as build

import importlib
import datetime
import re
import numpy as np
import pandas as pd


In [180]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [217]:
importlib.reload(build)
importlib.reload(config)

<module 'config' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/config.py'>

In [121]:
dataset_1 = pd.read_csv('../data/dataset_main_active1.csv')
dataset_3 = pd.read_csv('../data/dataset_main_active3.csv')

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [206]:
dataset_1.head(2)

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,PRIMARY_OFFENSE_CODE,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,felon_count,misd_count,Primary offense code_x,Description (if needed)_x,Recidivate_Risk_Level,Needed a check?_x,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Primary offense code_y,Description (if needed)_y,Current_Offense_Risk_Level,Needed a check?_y,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh,age_at_sentence,date_to_merge
0,4,AA,1983-07-12,SELL SCHEDULE II,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,SELL SCHEDULE II,NONE,NaN,0,0,NaN,0.0,0.0,2.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,SELL SCHEDULE II,Selling drugs on Schedule II - high potential ...,3.0,YES,2.0,4.0,21.0,1983-07
1,6,AA,1973-01-30,WORTHLESS CHECK,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,AB,1973-04-11,WORTHLESS CHECK,1973,0.0,1.0,0.0,0.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,21.0,1973-01


In [218]:
# BINARY TARGET VAR
grouping_target = config.grouping_target
print(grouping_target)
#print(features)
train_data, test_data, validate_data, active_sentences = build.split_and_process(dataset_1,config,grouping_target)


binary
249
Size of active sentences dataset:  32711
Dataset size:  778681


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_no_active[target_label] = dataset_no_active['label']


Total Number of Unique IDs: 419118
Total Number of IDs in Test Data: 83824
Total Number of IDs in Train Data: 268235
Total Number of IDs in Validate Data: 67059
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.20000095438516122
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.7999990456148388
Does Validate Represent 20% of the Train+Validate Data?: 0.20000059649143737
Does Train Represent 80% of the Train+Validate Data?: 0.7999994035085627
Index(['CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
    

In [219]:
# Sanity Check
print('Train and Test Check:')
build.sanity_check(train_data,test_data)

print('Train and Validate Check:')
build.sanity_check(train_data,validate_data)

print('Train and Active Check:')
build.sanity_check(train_data,active_sentences)



Train and Test Check:
Success: Features match
Success: No NAs remain
Train and Validate Check:
Success: Features match
Success: No NAs remain
Train and Active Check:
Success: Features match


In [221]:
train_data.groupby(target).count()

,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,CONTROL_STATUS_HIGH SECURITY/MAX CONTROL HCON,CONTROL_STATUS_INTENSIVE CONTROL ICON,CONTROL_STATUS_MAXIMUM CONTROL MCON,CONTROL_STATUS_MODIFIED HOUSING MODH,CONTROL_STATUS_OUT OF CELL RESTRICTION OUTC,CONTROL_STATUS_PROTECTIVE CONTROL PCON,CONTROL_STATUS_REGULAR POPULATION RPOP,CONTROL_STATUS_REST.HOUS. ADMINISTRATIVE RHAP,CONTROL_STATUS_REST.HOUS. DISCIPLINARY RHDP,CONTROL_STATUS_RESTRCT HOUSNG 4 CTL PURP RHCP,OFFENSE_CLEAN_ARMED ROBBERY,OFFENSE_CLEAN_ASSAULT BY STRANGULATION,OFFENSE_CLEAN_ASSAULT INFLICT SERI BODY INJ,OFFENSE_CLEAN_ASSAULT ISI,OFFENSE_CLEAN_ASSAULT ON FEMALE,OFFENSE_CLEAN_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_CLEAN_ASSAULT ON POLICEMAN,OFFENSE_CLEAN_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_CLEAN_AWDW,OFFENSE_CLEAN_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_CLEAN_AWDWISI,OFFENSE_CLEAN_AWDWWITKISI,OFFENSE_CLEAN_B & E & L,OFFENSE_CLEAN_B & E VEHICLES,OFFENSE_CLEAN_BURGLARY 1ST DEGREE,OFFENSE_CLEAN_BURGLARY 2ND DEGREE,OFFENSE_CLEAN_CHEAT - PROPERTY/SERVICES,OFFENSE_CLEAN_COMMON LAW FORGERY,OFFENSE_CLEAN_COMMON LAW ROBBERY,OFFENSE_CLEAN_COMMUNICATING THREATS,OFFENSE_CLEAN_CREDIT CARD THEFT,OFFENSE_CLEAN_DAMAGE TO PROPERTY,OFFENSE_CLEAN_DEL/SELL SCHEDULE II,OFFENSE_CLEAN_DISCHG FIREARM-OCC PROPERTY,OFFENSE_CLEAN_DRIV LICENSE PERM RVK,OFFENSE_CLEAN_DRIV LICENSE REVOKED,OFFENSE_CLEAN_DRIVING UNDER INFLUENCE (DUI),OFFENSE_CLEAN_DRUG PARA - USE/POSSESS,OFFENSE_CLEAN_DWI DRIVING WHILE IMPAIRED,OFFENSE_CLEAN_DWI LEVEL 1,OFFENSE_CLEAN_DWI LEVEL 1 AGGRAVATED,OFFENSE_CLEAN_DWI LEVEL 2,OFFENSE_CLEAN_DWI LEVEL 3,OFFENSE_CLEAN_DWI LEVEL 4,OFFENSE_CLEAN_DWI LEVEL 5,OFFENSE_CLEAN_EMBEZZLEMENT,OFFENSE_CLEAN_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_CLEAN_FELONY B&E,OFFENSE_CLEAN_FORGERY,OFFENSE_CLEAN_HABITUAL FELON,OFFENSE_CLEAN_HABITUAL IMPAIRED DRIVING,OFFENSE_CLEAN_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_CLEAN_IDENTITY FRAUD/THEFT,OFFENSE_CLEAN_INDECENT LIBERTY W/CHILD,OFFENSE_CLEAN_INVOLUNTARY MANSLAUGHTER,OFFENSE_CLEAN_KIDNAPPING 2ND DEGREE,OFFENSE_CLEAN_LARCENY,OFFENSE_CLEAN_LARCENY (OVER $200),OFFENSE_CLEAN_LARCENY AFTER B & E,OFFENSE_CLEAN_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_CLEAN_LARCENY FROM PERSON,OFFENSE_CLEAN_LARCENY OF FIREARMS,OFFENSE_CLEAN_LARCENY OF MOTOR VEHICLE,OFFENSE_CLEAN_LARCENY OVER $1000,OFFENSE_CLEAN_MAINT ANY PLACE CONTR SUB,OFFENSE_CLEAN_MANSLAUGHTER,OFFENSE_CLEAN_MISD B&E,OFFENSE_CLEAN_MURDER FIRST DEGREE,OFFENSE_CLEAN_MURDER SECOND DEGREE,OFFENSE_CLEAN_NON-SUPPORT,OFFENSE_CLEAN_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_CLEAN_OPERATE VEHICLE W/O LICENSE,OFFENSE_CLEAN_OTHER,OFFENSE_CLEAN_POSSESS SCHEDULE I,OFFENSE_CLEAN_POSSESS SCHEDULE II,OFFENSE_CLEAN_POSSESS SCHEDULE VI,OFFENSE_CLEAN_POSSESS WITS SCHEDULE I,OFFENSE_CLEAN_POSSESS WITS SCHEDULE II,OFFENSE_CLEAN_POSSESS WITS SCHEDULE VI,OFFENSE_CLEAN_POSSESSING STOLEN GOODS,OFFENSE_CLEAN_POSSESSION OF FIREARM BY FELON,OFFENSE_CLEAN_RAPE FIRST DEGREE,OFFENSE_CLEAN_RAPE SECOND DEGREE,OFFENSE_CLEAN_RECEIVING STOLEN GOODS,OFFENSE_CLEAN_RECEIVING STOLEN VEHICLE,OFFENSE_CLEAN_RESISTING OFFICER,OFFENSE_CLEAN_ROBBERY W/DANGEROUS WEAPON,OFFENSE_CLEAN_SELL SCHEDULE I,OFFENSE_CLEAN_SELL SCHEDULE II,OFFENSE_CLEAN_SELL SCHEDULE VI,OFFENSE_CLEAN_SEXUAL OFFENSE 2ND DEGREE,OFFENSE_CLEAN_SHOPLIFTING,OFFENSE_CLEAN_SPEED ELUDE ARREST/ATTEMPT/SEC,OFFENSE_CLEAN_SPEEDING ELUDE ARREST OR/ATTEM,OFFENSE_CLEAN_TRAFFICKING SCHEDULE I,OFFENSE_CLEAN_TRAFFICKING SCHEDULE II,OFFENSE_CLEAN_TRESPASS,OFFENSE_CLEAN_UNAUTH USE MOTOR CONVEYANCE,OFFENSE_CLEAN_UTTERING FORGEDPAPER/INST/END,OFFENSE_CLEAN_VIOLATION PROTECTIVE ORDER,OFFENSE_CLEAN_WORTHLESS CHECK,Current_Offense_Risk_Level_1.0,Current_Offense_Risk_Level_2.0,Current_Offense_Risk_Level_3.0,Current_Offense_Risk_Level_4.0,Current_Offense_Risk_Level_5.0,year_month_1976-01,year_month_1976-02,year_month_1976-03,year_month_1976-04,year_month_1976-05,year_month_1976-06,year_month_1976-07,year_month_1976-08,year_mont

In [222]:
#

In [223]:
#test_data.groupby(target).count()

# Analysis

In [229]:
MODELS = config.MODELS
PARAMS = config.PARAMS
model_key = "LogisticRegression"

model = MODELS[model_key]
params = PARAMS[model_key]        

target = config.target_vars[0]


In [230]:
params

{'penalty': ['l2', 'none'], 'C': [0.01, 0.1, 1, 10, 100], 'max_iter': [1000]}

In [231]:
target

'Recidivate_Risk_Level'

In [232]:
#https://stackoverflow.com/questions/31948879/using-explicit-predefined-validation-set-for-grid-search-with-sklearn
ps = PredefinedSplit(test_fold=validate_data[target])


In [233]:
#scorer = make_scorer(precision_score, average = None)
#gs_svc = GridSearchCV(estimator=svc_clf,param_grid=param_grid,scoring=scorer,cv=5)

scoring = {'recall': make_scorer(recall_score, average = 'macro'), 'precision': make_scorer(precision_score, average = 'macro'), 'accuracy': make_scorer(accuracy_score)}
grid_model = GridSearchCV(estimator=model, 
                          param_grid=params, 
                          cv=ps,
                          return_train_score=True,
                          scoring=scoring,
                             refit=False)



In [ ]:
grid_model_result = grid_model.fit(train_data.loc[:,train_data.columns!=target],train_data[target])


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore th

In [ ]:
# THREE CLASS VAR

In [ ]:
# 5 CLASS VAR

In [224]:
#model = LogisticRegression(random_state=1234, solver='lbfgs')

In [225]:
#model.fit(train_data.loc[:,train_data.columns!=target],train_data[target])

In [226]:
#validate_predict = model.predict(validate_data.loc[:,validate_data.columns!=target])

In [227]:
#accuracy = accuracy_score(validate_data[target],validate_predict)


In [228]:
#accuracy

In [184]:
print("Accuracy:\t{}\nPrecision:\t{}\nRecall:\t\t{}\nF1 Score:\t{}\n".format(accuracy_score(validate_data[target], validate_predict),
                                                                           precision_score(validate_data[target], validate_predict,average=None),
                                                                           recall_score(validate_data[target], validate_predict,average=None),
                                                                           f1_score(validate_data[target], validate_predict,average=None)
                                                                          ))

Accuracy:	0.8230410917313178
Precision:	[0.82725468 0.15324675 0.11285266 0.14035088 0.         0.        ]
Recall:		[0.9961088  0.00917431 0.00591522 0.00115274 0.         0.        ]
F1 Score:	[0.9038633  0.01731221 0.01124122 0.00228669 0.         0.        ]



/Users/daminisharma/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [164]:
#train_data.dtypes

In [196]:
test_data.groupby(target).count()

,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,RACE_ASIAN/ORTL,RACE_BLACK,RACE_INDIAN,RACE_OTHER,RACE_UNKNOWN,RACE_WHITE,GENDER_FEMALE,GENDER_MALE,ETHNICITY_AFRICAN,ETHNICITY_AMERICAN INDIAN,ETHNICITY_ASIAN,ETHNICITY_EUROPEAN/N.AM./AUSTR,ETHNICITY_HISPANIC/LATINO,ETHNICITY_NORDIC/SCANDANAVIAN,ETHNICITY_ORIENTAL,ETHNICITY_OTHER,ETHNICITY_PACIFIC ISLANDER,ETHNICITY_SLAVIC (E. EUROPEAN),ETHNICITY_UNKNOWN,CONTROL_STATUS_HIGH SECURITY/MAX CONTROL HCON,CONTROL_STATUS_INTENSIVE CONTROL ICON,CONTROL_STATUS_MAXIMUM CONTROL MCON,CONTROL_STATUS_MODIFIED HOUSING MODH,CONTROL_STATUS_OUT OF CELL RESTRICTION OUTC,CONTROL_STATUS_PROTECTIVE CONTROL PCON,CONTROL_STATUS_REGULAR POPULATION RPOP,CONTROL_STATUS_REST.HOUS. ADMINISTRATIVE RHAP,CONTROL_STATUS_REST.HOUS. DISCIPLINARY RHDP,CONTROL_STATUS_RESTRCT HOUSNG 4 CTL PURP RHCP,OFFENSE_CLEAN_ARMED ROBBERY,OFFENSE_CLEAN_ASSAULT BY STRANGULATION,OFFENSE_CLEAN_ASSAULT INFLICT SERI BODY INJ,OFFENSE_CLEAN_ASSAULT ISI,OFFENSE_CLEAN_ASSAULT ON FEMALE,OFFENSE_CLEAN_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_CLEAN_ASSAULT ON POLICEMAN,OFFENSE_CLEAN_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_CLEAN_AWDW,OFFENSE_CLEAN_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_CLEAN_AWDWISI,OFFENSE_CLEAN_AWDWWITKISI,OFFENSE_CLEAN_B & E & L,OFFENSE_CLEAN_B & E VEHICLES,OFFENSE_CLEAN_BURGLARY 1ST DEGREE,OFFENSE_CLEAN_BURGLARY 2ND DEGREE,OFFENSE_CLEAN_CHEAT - PROPERTY/SERVICES,OFFENSE_CLEAN_COMMON LAW FORGERY,OFFENSE_CLEAN_COMMON LAW ROBBERY,OFFENSE_CLEAN_COMMUNICATING THREATS,OFFENSE_CLEAN_CREDIT CARD THEFT,OFFENSE_CLEAN_DAMAGE TO PROPERTY,OFFENSE_CLEAN_DEL/SELL SCHEDULE II,OFFENSE_CLEAN_DISCHG FIREARM-OCC PROPERTY,OFFENSE_CLEAN_DRIV LICENSE PERM RVK,OFFENSE_CLEAN_DRIV LICENSE REVOKED,OFFENSE_CLEAN_DRIVING UNDER INFLUENCE (DUI),OFFENSE_CLEAN_DRUG PARA - USE/POSSESS,OFFENSE_CLEAN_DWI DRIVING WHILE IMPAIRED,OFFENSE_CLEAN_DWI LEVEL 1,OFFENSE_CLEAN_DWI LEVEL 1 AGGRAVATED,OFFENSE_CLEAN_DWI LEVEL 2,OFFENSE_CLEAN_DWI LEVEL 3,OFFENSE_CLEAN_DWI LEVEL 4,OFFENSE_CLEAN_DWI LEVEL 5,OFFENSE_CLEAN_EMBEZZLEMENT,OFFENSE_CLEAN_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_CLEAN_FELONY B&E,OFFENSE_CLEAN_FORGERY,OFFENSE_CLEAN_HABITUAL FELON,OFFENSE_CLEAN_HABITUAL IMPAIRED DRIVING,OFFENSE_CLEAN_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_CLEAN_IDENTITY FRAUD/THEFT,OFFENSE_CLEAN_INDECENT LIBERTY W/CHILD,OFFENSE_CLEAN_INVOLUNTARY MANSLAUGHTER,OFFENSE_CLEAN_KIDNAPPING 2ND DEGREE,OFFENSE_CLEAN_LARCENY,OFFENSE_CLEAN_LARCENY (OVER $200),OFFENSE_CLEAN_LARCENY AFTER B & E,OFFENSE_CLEAN_LARCENY BY SERVANT/EMPLOYEE,OFFENSE_CLEAN_LARCENY FROM PERSON,OFFENSE_CLEAN_LARCENY OF FIREARMS,OFFENSE_CLEAN_LARCENY OF MOTOR VEHICLE,OFFENSE_CLEAN_LARCENY OVER $1000,OFFENSE_CLEAN_MAINT ANY PLACE CONTR SUB,OFFENSE_CLEAN_MANSLAUGHTER,OFFENSE_CLEAN_MISD B&E,OFFENSE_CLEAN_MURDER FIRST DEGREE,OFFENSE_CLEAN_MURDER SECOND DEGREE,OFFENSE_CLEAN_NON-SUPPORT,OFFENSE_CLEAN_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_CLEAN_OPERATE VEHICLE W/O LICENSE,OFFENSE_CLEAN_OTHER,OFFENSE_CLEAN_POSSESS SCHEDULE I,OFFENSE_CLEAN_POSSESS SCHEDULE II,OFFENSE_CLEAN_POSSESS SCHEDULE VI,OFFENSE_CLEAN_POSSESS WITS SCHEDULE I,OFFENSE_CLEAN_POSSESS WITS SCHEDULE II,OFFENSE_CLEAN_POSSESS WITS SCHEDULE VI,OFFENSE_CLEAN_POSSESSING STOLEN GOODS,OFFENSE_CLEAN_POSSESSION OF FIREARM BY FELON,OFFENSE_CLEAN_RAPE FIRST DEGREE,OFFENSE_CLEAN_RAPE SECOND DEGREE,OFFENSE_CLEAN_RECEIVING STOLEN GOODS,OFFENSE_CLEAN_RECEIVING STOLEN VEHICLE,OFFENSE_CLEAN_RESISTING OFFICER,OFFENSE_CLEAN_ROBBERY W/DANGEROUS WEAPON,OFFENSE_CLEAN_SELL SCHEDULE I,OFFENSE_CLEAN_SELL SCHEDULE II,OFFENSE_CLEAN_SELL SCHEDULE VI,OFFENSE_CLEAN_SEXUAL OFFENSE 2ND DEGREE,OFFENSE_CLEAN_SHOPLIFTING,OFFENSE_CLEAN_SPEED ELUDE ARREST/ATTEMPT/SEC,OFFENSE_CLEAN_SPEEDING ELUDE ARREST OR/ATTEM,OFFENSE_CLEAN_TRAFFICKING SCHEDULE I,OFFENSE_CLEAN_TRAFFICKING SCHEDULE II,OFFENSE_CLEAN_TRESPASS,OFFENSE_CLEAN_UNAUTH USE MOTOR CONVEYANCE,OFFENSE_CLEAN_UTTERING FORGEDPAPER/INST/END,OFFENSE_CLEAN_VIOLATION PROTE

In [186]:
test_predict = model.predict(test_data.loc[:,test_data.columns!=target])

In [187]:
print("Accuracy:\t{}\nPrecision:\t{}\nRecall:\t\t{}\nF1 Score:\t{}\n".format(accuracy_score(test_data[target], test_predict),
                                                                           precision_score(test_data[target], test_predict,average=None),
                                                                           recall_score(test_data[target], test_predict,average=None),
                                                                           f1_score(test_data[target], test_predict,average=None)
                                                                          ))

Accuracy:	0.8275217932752179
Precision:	[0.82893432 0.34375    0.36981132 0.10638298 0.25       0.        ]
Recall:		[9.98557592e-01 5.67375887e-03 1.32647537e-02 5.78235226e-04
 3.95569620e-04 0.00000000e+00]
F1 Score:	[9.05873952e-01 1.11632627e-02 2.56108716e-02 1.15021854e-03
 7.89889415e-04 0.00000000e+00]



/Users/daminisharma/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
